In [1]:
import sys
import numpy as np
import pandas as pd
from virtualscada.vs import removeRows
from virtualscada.vs import fillValuesMLPFForward

ModuleNotFoundError: No module named 'virtualscada'

In [ ]:
sys.path.append('..')

In [ ]:
p = pd.read_csv('Data/output_p.csv', header = None)
q = pd.read_csv('Data/output_q.csv', header = None)
v = pd.read_csv('Data/output_v.csv', header = None)
a = pd.read_csv('Data/output_a.csv', header = None)